In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
from copy import deepcopy
import clin.eval.eval
import clin.eval.med_status
import clin.llm
import clin.parse
from collections import defaultdict
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'

sys.path.append('../experiments/')
results_dir = '../results/'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[r.dataset_name == 'medication_status']

# get data for eval
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
n = len(df)

r = clin.eval.med_status.add_status_eval(r, df)

In [15]:
r0 = r[(r.checkpoint == 'gpt-4-0314') * (r.seed == 1) * (r.n_shots == 5)].iloc[0]

In [16]:
r0

dataset_name                                                       medication_status
seed                                                                               1
save_dir                           /home/chansingh/clinical-extract-and-check/res...
checkpoint                                                                gpt-4-0314
checkpoint_verify                                                         gpt-4-0314
n_shots                                                                            5
use_cache                                                                          1
save_dir_unique                    /home/chansingh/clinical-extract-and-check/res...
extracted_strs                     ["Dobutamine" (active)\n- "Nipride" (active)\n...
recall___original                                                           0.908824
precision___original                                                        0.853591
f1___original                                                    

In [17]:
r0.columns

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 r0.columns                                                                                   │
│   2                                                                                              │
│                                                                                                  │
│ /home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/pandas/core/generic.py:5989 in       │
│ __getattr__                                                                                      │
│                                                                                                  │
│    5986 │   │   │   and self._info_axis._can_hold_identifiers_and_holds_name(name)               │
│    5987 │   │   ):                                                                               │
│    5988 │   │   │   return self[name]                                                            │
│ ❱  5989 │   │   return object.__getattribute__(self, name)                                       │
│    5990 │                                                                                        │
│    5991 │   def __setattr__(self, name: str, value) -> None:                                     │
│    5992 │   │   """                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Series' object has no attribute 'columns'

In [18]:
# display entire strings
with pd.option_context('display.max_colwidth', None):
    display(r0[['extracted_strs']])

extracted_strs    ["Dobutamine" (active)\n- "Nipride" (active)\n- "Esmolol" (discontinued), "alcohol withdrawal protocol" (discontinued)\n- "Ativan" (active)\n- "Inderal LA" (active)\n- "Celexa" (neither)\n- "Lexapro" (active), "Citrucel" (active)\n- "Colace" (discontinued)\n- "Dulcolax" (active)\n- "Senokot S" (active), "Lovenox" (active)\n- "aspirin" (on hold)\n- "Coumadin" (on hold)\n- "Lopressor" (on hold)\n- "Lasix" (on hold)\n- "spironolactone" (on hold)\n- "metoprolol" (on hold)\n- "glyburide" (on hold), "linezolid" (active)\n- "Levaquin" (active)\n- "Bactrim" (active)\n- "vancomycin" (discontinued), "lisinopril" (active)\n- "Questran" (discontinued)\n- "nebs" (active), "Ambien" (active)\n- "Vicodin" (active)\n- "Noroxin" (active)\n- "Flagyl" (discontinued)\n- "Lasix" (discontinued), "vancomycin" (active)\n- "Tequin" (discontinued)\n- "Penicillin V" (considered for future use), "lactulose" (active)\n- "rifampin" (active)\n- "Inderal-S" (discontinued), "acyclovir" (discontinued)\

In [19]:
for s in r0['extracted_strs']:
    print(s)
    print()

"Dobutamine" (active)
- "Nipride" (active)
- "Esmolol" (discontinued)

"alcohol withdrawal protocol" (discontinued)
- "Ativan" (active)
- "Inderal LA" (active)
- "Celexa" (neither)
- "Lexapro" (active)

"Citrucel" (active)
- "Colace" (discontinued)
- "Dulcolax" (active)
- "Senokot S" (active)

"Lovenox" (active)
- "aspirin" (on hold)
- "Coumadin" (on hold)
- "Lopressor" (on hold)
- "Lasix" (on hold)
- "spironolactone" (on hold)
- "metoprolol" (on hold)
- "glyburide" (on hold)

"linezolid" (active)
- "Levaquin" (active)
- "Bactrim" (active)
- "vancomycin" (discontinued)

"lisinopril" (active)
- "Questran" (discontinued)
- "nebs" (active)

"Ambien" (active)
- "Vicodin" (active)
- "Noroxin" (active)
- "Flagyl" (discontinued)
- "Lasix" (discontinued)

"vancomycin" (active)
- "Tequin" (discontinued)
- "Penicillin V" (considered for future use)

"lactulose" (active)
- "rifampin" (active)
- "Inderal-S" (discontinued)

"acyclovir" (discontinued)
- "TMP-SMX" (discontinued)
- "Pentamidine" (acti

In [ ]:
# print errs for med extraction
# mets = clin.eval.med_status.calculate_metrics(row['dict_ov_pv_ev'], df, verbose=True)

# print errors for status extraction
# accs_cond, f1s_macro_cond = clin.eval.eval_medication_status([med_status_dicts_list[-1]], df, verbose=True)

## Interpret an example

In [ ]:
# single row to investigate
row = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[0]

In [ ]:
i = 11
ev = row['dict_evidence_ov_pv_ev'][i]
med_status_dict = row['dict_sv'][i]
gt = df.iloc[i]
print(gt)
print(med_status_dict)
print('\n' + gt['snippet'])

for k in ev:
    print(k)
    print('\t', ev[k])
    print()